In [ ]:
%python -m pip install pydantic

  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   -------------------------------- ------- 1.6/1.9 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 5.5 MB/s eta 0:00:00
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)


In [8]:
from pydantic import BaseModel

class User(BaseModel):
    name:str = 'john'
    id:int

In [9]:
user = User(id = "123")
print(user)

name='john' id=123


In [10]:
print(user.model_fields_set)
user = User(name = "Poky", id = "789")
print(user.model_fields_set)

{'id'}
{'name', 'id'}


In [11]:
print(user.model_dump())
print(user.model_dump_json())
print(user.model_json_schema())

{'name': 'Poky', 'id': 789}
{"name":"Poky","id":789}
{'properties': {'name': {'default': 'john', 'title': 'Name', 'type': 'string'}, 'id': {'title': 'Id', 'type': 'integer'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


Nested Models

In [13]:
from typing import List, Optional
from pydantic import BaseModel

class Food(BaseModel):
    name : str
    price : float
    ingredients : Optional [List[str]] = None

class Restaurant(BaseModel):
    name : str
    location : str
    foods : List[Food]


res_instance  = Restaurant(
    name = "social",
    location = "123, Kormangala",
    foods=[
        {"name" : "Cheese Pizza", "price":12.34, "ingredients":["cheese","Flour"]},
        {"name":"Burger","price":8.9}
    ]
)   
print(res_instance)
print(res_instance.model_dump())


name='social' location='123, Kormangala' foods=[Food(name='Cheese Pizza', price=12.34, ingredients=['cheese', 'Flour']), Food(name='Burger', price=8.9, ingredients=None)]
{'name': 'social', 'location': '123, Kormangala', 'foods': [{'name': 'Cheese Pizza', 'price': 12.34, 'ingredients': ['cheese', 'Flour']}, {'name': 'Burger', 'price': 8.9, 'ingredients': None}]}


In [16]:
%pip install pydantic[email]

  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
from typing import List
from pydantic import BaseModel, EmailStr, PositiveInt,conlist, Field, HttpUrl

class Address(BaseModel):
    street : str
    city : str
    state : str
    zip_code : str

class Employee(BaseModel):
    name : str
    position : str
    email : EmailStr

class Owner(BaseModel):
    name : str
    email : EmailStr

class Restaurant(BaseModel):
    name : str = Field(..., pattern= r"^[a-zA-Z0-9-' ]+$")
    owner : Owner
    address : Address
    employees : conlist(Employee, min_length=2)
    number_of_seats: PositiveInt
    delivery: bool
    website : HttpUrl


# creating an instance of the restaurant class
res_instance = Restaurant(
    name = "Social",
    owner={
        "name" :"John",
        "email" : "abc@gmail.com"
    },
    address={
        "street":"123,Paris",
        "city":"London",
        "state":"Washington",
        "zip_code":"12344"
    },
    employees=[
        {
            "name":"Rita",
            "position":"HR",
            "email" : "rita@google.com"
        },
        {
            "name":"sita",
            "position":"chef",
            "email" : "sita@google.com"
        }
    ],
    number_of_seats=50,
    delivery=True,
    website="http://social.com"
)    
#  printing instance
print(res_instance)
print(res_instance.model_dump)



name='Social' owner=Owner(name='John', email='abc@gmail.com') address=Address(street='123,Paris', city='London', state='Washington', zip_code='12344') employees=[Employee(name='Rita', position='HR', email='rita@google.com'), Employee(name='sita', position='chef', email='sita@google.com')] number_of_seats=50 delivery=True website=Url('http://social.com/')
<bound method BaseModel.model_dump of Restaurant(name='Social', owner=Owner(name='John', email='abc@gmail.com'), address=Address(street='123,Paris', city='London', state='Washington', zip_code='12344'), employees=[Employee(name='Rita', position='HR', email='rita@google.com'), Employee(name='sita', position='chef', email='sita@google.com')], number_of_seats=50, delivery=True, website=Url('http://social.com/'))>


In [9]:
from typing import List
from pydantic import BaseModel, conlist, HttpUrl

class Student(BaseModel):
    name : str
    id : int
    fee : float

class Teacher(BaseModel):
    name : str
    Class : bool

class Class(BaseModel):
    student : Student
    teacher : Teacher
    no_of_sections : int

# creaating an instance of class
class_insatnce = Class(
    student = {
        "name":"varsha",
        "id" : 123,
        "fee": 1200
    },
    teacher = {
        "name":"Supriya",
        "Class": False
    },
    no_of_sections = 4
)         
print(class_insatnce)
print(class_insatnce.model_dump)

student=Student(name='varsha', id=123, fee=1200.0) teacher=Teacher(name='Supriya', Class=False) no_of_sections=4
<bound method BaseModel.model_dump of Class(student=Student(name='varsha', id=123, fee=1200.0), teacher=Teacher(name='Supriya', Class=False), no_of_sections=4)>


Field Validation

In [16]:
from pydantic import BaseModel, EmailStr, field_validator

class Owner(BaseModel):
    name : str
    email : EmailStr

    @field_validator('name')
    @classmethod
    def name_must_contain_space(cls, v: str) -> str:
        if ' ' not in v:
            raise ValueError('owner name must contain a space')
        # return v.title()
        # return v.upper()
        return v.lower()
    
try:
    Owner_instance = Owner(name = "faBI roCk", email="abc@gmail.com")
    # Owner_instance = Owner(name = "fabi rock", email="abc@gmail.com")

except ValueError as e:
    print(e)  

print(Owner_instance)          

name='fabi rock' email='abc@gmail.com'


Model validators - allow to create a model before and after field validation

In [24]:
from typing import Any
from pydantic import BaseModel, EmailStr, ValidationError,model_validator

class Owner(BaseModel):
    name : str
    email : EmailStr

    @model_validator(mode = 'before')
    @classmethod
    def check_sensitive_info_omitted(cls, data: Any) -> Any:
        if isinstance(data, dict):
            if 'password' in data:
                raise ValueError('password should not be included')
            if 'card_number' in data:
                raise ValueError('card_number should not be included')

        return data
    @model_validator(mode = 'after')
    def check_name_contains_space(self) -> 'Owner':
        if ' ' not in self.name:
            raise ValueError('Owner name must contain a space')
        return self
    
print(Owner(name="John Doee", email="ab@em.com"))

try:
    # Owner(name="John Doee", email="ab@em.com")
     Owner(name="John Doee", email="ab@em.com", password = "12pass")
except ValueError as e:
    print(e)   


name='John Doee' email='ab@em.com'
1 validation error for Owner
  Value error, password should not be included [type=value_error, input_value={'name': 'John Doee', 'em...', 'password': '12pass'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


Fields - Used to customize and ass metadata to fields of models

In [25]:
from pydantic import BaseModel, Field

class User(BaseModel):
    name : str = Field(default='john doe')

user = User()
print(user)

name='john doe'


Creating dynamic id using uuid4

In [27]:
from pydantic import BaseModel, Field
from uuid import uuid4

class User(BaseModel):
    id : int = Field(default_factory=lambda: uuid4().hex)

user = User()
print(user)

id='939c7f20e64e46d3bedb1d59b9472bac'


Field aliases - For validation and serialization, alias can be used

There are three ways to define an alias:
> Field(..., alias = 'foo')
> Field(..., validation_alias = 'foo')
> Filed(..., serialization_alias= 'foo')

In [ ]:
from pydantic import BaseModel, Field

class User(BaseModel):
    name : str = Field(..., alias = 'username')

user = User(username='jack')
print(user)
# print(user.model_dump(by_alias=False))
print(user.model_dump(by_alias=True))




name='jack'
{'name': 'jack'}
{'username': 'jack'}


In [38]:
from typing import List
from pydantic import BaseModel, Field, EmailStr

class User(BaseModel):
    username : str = Field(..., min_length=3, max_length=10)
    email: EmailStr = Field(...)
    age: int = Field(..., gt =0, le=120)
    height: float = Field(..., gt=0.0)
    is_active: bool = Field(True)

user_instance = User(
    username = "rgfefrer",
    email = "abd@gn.com",
    age = 30,
    height= 8.9,
    # is_active=False
)    
print(user_instance.model_dump())

{'username': 'rgfefrer', 'email': 'abd@gn.com', 'age': 30, 'height': 8.9, 'is_active': True}


Computed Fields

In [ ]:
from pydantic import BaseModel, ValidationError, field_validator
from datetime import datetime

class Person(BaseModel):
    name : str
    birth_year : int

    @property
    def age(self) -> int:
        current_year = datetime.now().year
        return current_year
    
    @field_validator('birth_year')
    @classmethod
    def validate(cls, v:int) -> int:
        current_year = datetime.now().year
        if current_year - v < 18:
            raise ValueError("Person must be 18 years or older")
        return v
try:
    # print(Person(name = "abhi",birth_year = 2000))
    print(Person(name = "abhi",birth_year = 2018))
except ValidationError as e:
    print(e)    



1 validation error for Person
birth_year
  Value error, Person must be 18 years or older [type=value_error, input_value=2018, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


Strict Mode 

In [44]:
from pydantic import BaseModel, ValidationError # type: ignore

class User(BaseModel):
    id : int
    name : str

print(User.model_validate({'id':'42','name':'john_doe'}))    
    

id=42 name='john_doe'


In [46]:
try:
    User.model_validate({'id':'42', 'name':'honk_tom'}, strict= True)
except ValidationError as exc:
    print(exc)    

1 validation error for User
id
  Input should be a valid integer [type=int_type, input_value='42', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_type


Settings Management

In [47]:
%pip install pydantic-settings

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from pydantic import Field
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    auth_key: str = Field(...)
    api_key: str = Field(alias='my_api_key')

print(Settings().model_dump())    

{'auth_key': 'env_auth_key', 'api_key': 'env_test'}


In [2]:
import os
from pydantic import Field, AliasChoices
from pydantic_settings import BaseSettings

os.environ["AUTH_KEY"] = "test_auth_key"
os.environ["MY_API_KEY"] = "test"
os.environ["ENV2"] = "https://mysuperurl.com"

class Settings(BaseSettings):
    service_name: str = Field(default="default")
    auth_key: str
    api_key: str = Field(alias="my_api_key")
    url: str = Field(validation_alias=AliasChoices("env1", "env2"))

print(Settings().model_dump())


{'service_name': 'default', 'auth_key': 'test_auth_key', 'api_key': 'test', 'url': 'https://mysuperurl.com'}


In [3]:
import os
from pydantic import Field
from pydantic_settings import BaseSettings, SettingsConfigDict

# Set environment variables with the prefix
os.environ["PRODUCTION_AUTH_KEY"] = "test_auth_key"
os.environ["PRODUCTION_MY_API_KEY"] = "test"
os.environ["PRODUCTION_ENV2"] = "https://mysuperurl.com"

class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_prefix="production_")

    service_name: str = Field(default="default")
    auth_key: str
    api_key: str = Field(alias="my_api_key")
    url: str = Field(validation_alias=AliasChoices("env1", "env2"))

print(Settings().model_dump())


{'service_name': 'default', 'auth_key': 'test_auth_key', 'api_key': 'test', 'url': 'https://mysuperurl.com'}


Working with .env file

In [5]:
from pydantic_settings import BaseSettings, SettingsConfigDict
from pydantic import Field

class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_file='.env', env_file_encoding='utf-8')

    service_name: str = Field(default="default")
    auth_key: str
    api_key: str = Field(alias='my_api_key')

print(Settings().model_dump())


{'service_name': 'default', 'auth_key': 'test_auth_key', 'api_key': 'test'}


In [8]:
from pydantic_settings import BaseSettings, SettingsConfigDict
from pydantic import Field

class Settings(BaseSettings):
    # model_config = SettingsConfigDict(env_file='.env', env_file_encoding='utf-8', extra="allow")
    model_config = SettingsConfigDict(env_file='.env', env_file_encoding='utf-8', extra="ignore")


    service_name: str = Field(default="default")
    auth_key: str
    api_key: str = Field(alias='my_api_key')

print(Settings().model_dump())


{'service_name': 'default', 'auth_key': 'test_auth_key', 'api_key': 'test'}
